<font color='red'>
<br>Content:
    
* [Introduction](#1)
* [Overview the Data Set](#2)
* [Logistic Regression](#3)
    * [Initializing parameters](#4)
        * Sigmoid Function
    * [Optimization Algorithm with Gradient Descent](#5)
        * Forward & Backward Propagation
        * Loss(error) Function
        * Cost Function
        * Updating parameters
        * Comparison of Logistic Regression Results
    * [KNN Algorithm](#6)
* [Conclusion](#7)

<a id="1"></a> <br>
# Introduction

* This dataset include estimation of rain for tomorrow in Austrilia. In this work, tomorrow rain possibility (1 or 0) results will learn then testing with logistic regression. After that, KNN Classification will apply to data to find out KNN Score and number of neighbors.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import warnings
# filter warnings
warnings.filterwarnings('ignore')
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<a id="2"></a> <br>
# Overview the Data Set

* Firstly we will start to understand data. Therefore, we will check data columns and types.

In [ ]:
df = pd.read_csv("../input/weather-dataset-rattle-package/weatherAUS.csv")
df.head()
df.info()

* As you can see our dataset is occurred with 23 columns. These columns types can define as float and object. However, in this work we need to relationship data values that is occurred as float. Beucause of that some columns that have object data types will drop in this part to obtain train and test data frames. After the dropping, we will drop NaN values from to data. But, if you want you can fill NaN values with mean values of each columns.

In [ ]:

df.drop(["Date", "Location","WindDir9am","WindDir3pm", "WindGustDir", "RainToday"], axis = 1, inplace = True)
df.dropna(inplace=True)
df.info()


* In this work we need to two results for logictic regression in variable. This variable is called as "RainTomorrow in this dataset. However, RainTomorrow columns are including "Yes" or "No". To apply logistic regression and KNN classification to data, we have to change our outputs with 0 or 1 to convert results as integer
* Yes = 1
* No = 0

In [ ]:
df.RainTomorrow = [1 if each == "Yes" else 0 for each in df.RainTomorrow]
df.RainTomorrow.value_counts()

In [ ]:
df.info()

<a id="3"></a> <br>
# Logistic Regression

<a id="4"></a> <br>
## Initilazing Parameters

* Our aim to split result data and other datas that effect to result data. 

In [ ]:
y = df.RainTomorrow.values
x_data = df.drop(["RainTomorrow"], axis = 1)


In [ ]:
# Lets normalize the dataset to pretend higher scaling factor between data features.

x = (x_data - np.min(x_data))/(np.max(x_data) - np.min(x_data)).values

* Now, to learn our data we have to split dataset as train and test. Test data will be occured %20 of the all data.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 42)

x_train = x_train.T
y_train = y_train.T
x_test = x_test.T
y_test = y_test.T

print("x_train:", x_train.shape)
print("y_train:", y_train.shape)
print("x_test:", x_test.shape)
print("y_test:", y_test.shape)

* We know Python has library to find accuracy of the test data. However, firstly we want to find accuracy wind some functions that write by hand calculation methods.

### Sigmoid Function

In [ ]:
def initialize_weigths_and_bias(dimension):
    w = np.full((dimension,1),0.01)
    b = 0.0
    return w,b

w,b = initialize_weigths_and_bias(5) # example


In [ ]:
def sigmoid(z):
    y_head = 1/(1 + np.exp(-z))
    return y_head

y_head = sigmoid(0.9) # example
y_head

<a id="5"></a> <br>
## Optimization Algorithm with Gradient Descent

* Now we will train our data with Forward and Backward propagation. It will help to update our data to find sufficent slope of the sigmoid function. If slope will going to 0, that means our data will learn as much as correctly. 

### Forward and Backward Propagation

In [ ]:
def forward_backward_propagation(w,b,x_train,y_train):
    #forward propagation
    z = np.dot(w.T,x_train) + b
    y_head = sigmoid(z)
    loss = -y_train*np.log(y_head)-(1-y_train)*np.log(1-y_head)
    cost = (np.sum(loss))/x_train.shape[1]              # x_train.shape[1] is for scailing
    
    #backward propagation
    derivative_weight = (np.dot(x_train, ((y_head-y_train).T)))/x_train.shape[1]
    derivative_bias = np.sum(y_head-y_train)/x_train.shape[1]
    gradients = {"derivative_weight": derivative_weight, "derivative_bias": derivative_bias}
    return cost,gradients



### Loss & Cost Function & Update Parameters

In [ ]:
def update(w,b,x_train,y_train,learning_rate, number_of_iteration):
    cost_list = []
    cost_list2 = []
    index = []
    
    #updating learning parameters is number of iteration times
    
    for i in range(number_of_iteration):
        # make forward and backward propagation and find cost and gradients
        cost,gradients = forward_backward_propagation(w,b,x_train,y_train)
        cost_list.append(cost)
        
        #lets update
        w = w - learning_rate * gradients["derivative_weight"]
        b = b- learning_rate * gradients["derivative_bias"]
        
        if i % 10  == 0:
            cost_list2.append(cost)
            index.append(i)
            print("Cost after iteration %i: %f" %(i,cost))
    # we update(learn) parameters weigths and bias
    parameters = {"weight": w, "bias": b}
    plt.plot(index,cost_list2)
    plt.xticks(index,rotation='vertical')
    plt.xlabel("Number of Iterarion")
    plt.ylabel("Cost")
    plt.show()
    return parameters, gradients, cost_list



* Now we define the function to check prediction of the test data.

In [ ]:
def predict(w,b,x_test):
    
    z = sigmoid(np.dot(w.T,x_test)+b)
    y_predict = np.zeros((1,x_test.shape[1]))
    
    for i in range(z.shape[1]):
        if z[0,i] <= 0.5:
            y_predict[0,i] = 0
        else:
            y_predict[0,i] = 1
            
    return y_predict

* Finally we reach to logistic regression function. Now, this function will learn our dataset with the other functions that we defined above. 

In [ ]:
def logistic_regression(x_train,y_train,x_test,y_test,learning_rate,num_iterations):
    dimension = x_train.shape[0]
    w,b = initialize_weigths_and_bias(dimension)
    
    parameters, gradients, cost_list = update(w,b,x_train,y_train,learning_rate,num_iterations)
    y_predict_test = predict(parameters["weight"], parameters["bias"], x_test)
    
    print("test accuracy: {} %".format(100 - np.mean(np.abs(y_predict_test - y_test)) * 100))
    
logistic_regression(x_train,y_train,x_test,y_test,learning_rate = 3,num_iterations=1000)

* Lets find our test accuracy with Python LogisticRegression method that based on sklearn library.

In [ ]:

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(x_train.T, y_train.T)

print("test accuracy {}".format(lr.score(x_test.T,y_test.T)))

* As you can see, our test accuracy results that found out with personal functions and LogisticRegression method.is so close.

<a id="6"></a> <br>
## KNN Algorithm


In [ ]:
# KNN Algorithm of Data

x_train = x_train.T
y_train = y_train.T
x_test = x_test.T
y_test = y_test.T

In [ ]:
print("x_train:", x_train.shape)
print("y_train:", y_train.shape)
print("x_test:", x_test.shape)
print("y_test:", y_test.shape)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 25)

knn.fit(x_train,y_train)
prediction = knn.predict(x_test)

print(" {} nn score: {}".format(25,knn.score(x_test,y_test)))



In [ ]:
# How can we find best number of neighbors with for loop

score_list = []

for each in range(1,25):
    knn2 = KNeighborsClassifier(n_neighbors = each)
    knn2.fit(x_train,y_train)
    score_list.append(knn2.score(x_test,y_test))
    
plt.plot(range(1,25), score_list)
plt.xlabel("k values")
plt.ylabel("accuracy")
plt.show()


<a id="7"></a> <br>
# Conclusion

* Our functional and LogisticRegression method results is find so close each other. We just present functional method to understand and show of the Logistic Regression methodology. After the this work, we can use LogisticRegression method that based on sklearn libray.

* KNN Algorithm is applied with KNeighborsClassifier method. In this part, we have to decide number of neighbors of the algorithm. For this that, we used for loop to find the best number of neighbors that is given the best accuracy result.

* The other Machine Learning and Deep Learning methodology will use in this notebook. 